In [2]:
from pipe import *
from functools import partial

from research.retina.detector import *
from research.optical_flow.focused import *
from research.optical_flow.dense import *
from research.deep_pipeline.models import *
from research.optical_flow.utils import *
from research.deep_pipeline.utils import combine_models

from utils.image import imprint

import os
import keras
import json
import pickle

import mPyPl as mp
from mPyPl.utils.flowutils import *

In [4]:
# Load config dictionary
config = pickle.load(open('config.pickle', 'rb'))

# RetinaFlow setup
retina = RetinaNet(config['retinaflow']['path'])
flow = FocusedFlow(
    config['retinaflow']['edge_detector'], 
    config['retinaflow']['lucas_kanade']
)

# VGG16 embeddings setup
vgg = keras.applications.vgg16.VGG16(
    include_top=False, 
    weights='imagenet', 
    input_shape=config['embeddings']['input_shape']
)

/anaconda/envs/py35/lib/python3.5/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
# Combined model setup
m1 = RetinaFlowNN(config['retinaflow']['shape'])
m2 = EmbeddingsNN(config['embeddings']['output_shape'])
m3 = DenseFlowNN(config['denseflow']['shape'])
model = combine_models(
    [
        config['retinaflow']['shape'], 
        config['embeddings']['output_shape'], 
        config['denseflow']['shape']
    ], 
    [m1, m2, m3]
)
model.load_weights(config['combined']['path'])
model.summary()

race-events-recognition/research/deep_pipeline/utils.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`
  model = Model(input=inputs,output=output)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 190, 100, 2)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 125, 7680, 1) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 190, 100, 2)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 1)            952577      input_2[0][0]                    
__________________________________________________________________________________________________
sequential

In [8]:
%%time

output_file = 'videos/output.mp4'

stream = (
    mp.videosource_chunked('videos/output2.mp4',video_size=config['retinaflow']['video_size'])
    | mp.chunk_slide(5)
    | mp.as_field('video')
    | mp.apply('video', 'uint_video', lambda x: x.astype(np.uint8))
    # RetinaFlow pipeline
    | mp.apply('uint_video', 'boxes', lambda x: [retina.predict(frame) for frame in x])
    | mp.apply(['uint_video', 'boxes'], 'raw_rflows', 
               lambda args: flow.run(args[0], args[1], config['retinaflow']['offset']))
    | mp.apply('raw_rflows', 'gradients', calc_gradients)
    | mp.apply('gradients', 'polar', to_polar)
    | mp.apply('polar', 'histograms', partial(video_to_hist,params=config['retinaflow']['hist_params']))
    | mp.apply('histograms', 'res_rflows', partial(zero_pad_hist,shape=config['retinaflow']['shape']))
    | mp.delfield(['boxes', 'gradients', 'raw_rflows', 'polar', 'histograms'])
    # DenseFlow pipeline
    | mp.apply('uint_video', 'dflow_video', partial(resize_video,video_size=config['denseflow']['video_size']))
    | mp.apply('dflow_video', 'raw_dflows', get_flow_descriptor)
    | mp.apply('raw_dflows','padded', partial(mp.zero_pad,max_frames=config['max_frames']))
    | mp.apply('padded','res_dflows', partial(
        normalize_histograms,
        ang_diap=config['denseflow']['ang_diap'],
        mag_diap=config['denseflow']['mag_diap']
    ))
    | mp.delfield(['dflow_video', 'raw_dflows', 'padded'])
    # VGG16 embeddings pipeline
    | mp.apply('video', 'vgg_video', partial(resize_video,video_size=config['embeddings']['video_size']))
    | mp.apply('vgg_video', 'res_vgg', 
               lambda x: vgg.predict(keras.applications.vgg16.preprocess_input(x)).reshape(config['embeddings']['output_shape']))
    | mp.delfield('vgg_video')
    # Utils
    | mp.apply('res_rflows', 'rflows', partial(np.expand_dims,axis=0))
    | mp.apply('res_dflows', 'dflows', partial(np.expand_dims,axis=0))
    | mp.apply('res_vgg', 'vgg', partial(np.expand_dims,axis=0))
    | mp.delfield(['res_rflows', 'res_dflows', 'res_vgg'])
    # Predict
    | mp.apply(['rflows', 'vgg', 'dflows'],'score', lambda args: model.predict(args))
    | mp.apply(['video','score'],'video_to_display',lambda args: imprint(args[0][50:75],args[1:]))
    | mp.select_field('video_to_display')
    | mp.collect_video(output_file)
)

CPU times: user 18.2 s, sys: 3.5 s, total: 21.7 s
Wall time: 52.1 s
